In [ ]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns

import warnings
warnings.filterwarnings("ignore")
from myutils.process_utils import process_linedf, post_process, process_timetable
from myutils.analysis_utils import load_daydf, load_routinedf, map_plot



In [ ]:
day

In [ ]:
date = 7
nidx = 0
direction = 1

In [ ]:
# for date in ['06','07','08','09','10']:
#     up_res = pd.DataFrame()
#     down_res = pd.DataFrame()
#     gps = load_daydf(int(date))
#     for nidx in gps.nidx.unique():
#         print(nidx)
#         for direction in [0, 1]:
#             routine = load_routinedf(gps, nidx, direction)
#             # if direction == 0:
#             #     if ''routine['station_status'][0]
#             routine = post_process(routine, date=int(date))
#             routine = routine[['vid','time','direction','nidx', 
#                                 'deadheading','cum_length', 'velocity','station_status']]
#             base_data = routine.drop('time', axis = 1).groupby(['nidx','direction','station_status']).agg('mean')
#             time_data = routine.groupby(['nidx','direction','station_status'])['time'].agg(['min','max'])
#             routine = pd.merge(base_data, time_data,left_index=True, right_index=True).sort_values('max').reset_index()
#             if direction == 0:
#                 up_res = pd.concat([up_res, routine], ignore_index=True).reset_index(drop = True)
#             else:
#                 down_res = pd.concat([down_res, routine], ignore_index=True).reset_index(drop = True)
#     up_res.to_pickle('./data/analysis/'+'truetime_up_09'+date+'.pkl')
#     down_res.to_pickle('./data/analysis/'+'truetime_down_09'+date+'.pkl')

In [ ]:
# ass = pd.read_pickle('./data/analysis/as_6.pkl')
# base_data = ass.drop('time', axis = 1).groupby(['nidx','direction','station_status']).agg('mean')
# time_data = ass.groupby(['nidx','direction','station_status'])['time'].agg(['min','max'])
# ass = pd.merge(base_data, time_data,left_index=True, right_index=True)
# ass.reset_index()
# # ass.groupby()
# ass.groupby()


In [ ]:
# new_ass = pd.read_pickle('./data/analysis/truetime_up_0906.pkl')
# new_ass.set_index(['station_status']).loc['菊园车站']

In [ ]:
day = load_daydf(date)
routine = load_routinedf(day, nidx, direction)
mapline = process_linedf(direction)
xlim = (7800+1.349e7, 8700+1.349e7)
ylim = (3.679e6, 3.682e6)
map_plot(mapline, routine)
map_plot(mapline, routine, xlim, ylim)

station_time_res = routine.loc[routine['station_status'] != 0, ['time','cum_length', 'station_status']].groupby('station_status').agg('mean').sort_values('time')
len(station_time_res)


In [ ]:
print(day.time.min(), day.time.max()) 
fig, ax = plt.subplots(dpi=300, figsize = (15,6))
for nidx in day.nidx.unique():
    routine = load_routinedf(day, nidx, direction)
    routine = post_process(routine, date=date)
    if len(routine) > 0:
        if routine.deadheading[0] == 0:
            ax.plot(routine.time, routine.cum_length, color = 'green')
        else:
            ax.plot(routine.time, routine.cum_length, color = 'red')
        
# ax.plot(routine1.time, routine1.cum_length)
# ax.plot(routine2.time, routine2.cum_length)

1. 分析首末班车发车准点率

In [ ]:
# start_time_lolim = timetable_up['start_time_lolim']
# start_time_hilim = timetable_up['start_time_hilim']
# maxtime = truetime_up['max'][0]
# start_tar = timetable_up.loc[
#             (
#                 (start_time_lolim <= maxtime) 
#                 & (start_time_hilim >= maxtime)
#             )
#     ].reset_index(drop = True)
# start_tar['start_time_lolim']

In [ ]:
from datetime import datetime as dt

def time_matching(truetime_row, timetabledf):
    mintime = truetime_row['min']
    maxtime = truetime_row['max']
    start_time_lolim = timetabledf['start_time_lolim']
    start_time_hilim = timetabledf['start_time_hilim']
    end_time_lolim = timetabledf['end_time_lolim']
    end_time_hilim = timetabledf['end_time_hilim']
    
    start_tar = timetabledf.loc[
                (
                    (start_time_lolim <= maxtime) 
                    & (start_time_hilim >= maxtime)
                )
        ].reset_index(drop = True)
    end_tar = timetabledf.loc[
                (
                    ( end_time_lolim <= mintime) 
                    & ( end_time_hilim >= mintime)
                )
        ].reset_index(drop = True)
    
    if truetime_row['direction'] == 0:
        if truetime_row['station_status'] == '菊园车站':
            if len(start_tar) > 0:
                truetime_row['intime'] = True
                truetime_row['lolim'] = start_tar['start_time_lolim'][0]
                truetime_row['hilim'] = start_tar['start_time_hilim'][0]
        else:
            if len(end_tar) > 0:
                truetime_row['intime'] = True
                truetime_row['lolim'] = end_tar['end_time_lolim'][0]
                truetime_row['hilim'] = end_tar['end_time_lolim'][0]
    else:
        if truetime_row['station_status'] == '公交嘉定新城站':
            if len(start_tar) > 0:
                truetime_row['intime'] = True
                truetime_row['lolim'] = start_tar['start_time_lolim'][0]
                truetime_row['hilim'] = start_tar['start_time_hilim'][0]
        else:
            if len(end_tar) > 0:
                truetime_row['intime'] = True
                truetime_row['lolim'] = end_tar['end_time_lolim'][0]
                truetime_row['hilim'] = end_tar['end_time_lolim'][0]
    
    return truetime_row


def generate_intime(date='06', time_range=[0, 0, 23, 59]):
    # for date in ['06','07','08','09','10']:
    lolim = dt(2021, 9 , int(date), hour=int(time_range[0]),minute=int(time_range[1]),second=0)
    hilim = dt(2021, 9 , int(date), hour=int(time_range[2]),minute=int(time_range[3]),second=0)
    
    path_timetable = './data/timetable/'
    timetable_up = pd.read_csv(path_timetable + 'timetable_up_09' + date + '.csv')
    timetable_up = process_timetable(timetable_up, int(date), strict = False)
    timetable_down = pd.read_csv(path_timetable + 'timetable_down_09' + date + '.csv')
    timetable_down = process_timetable(timetable_down, int(date), strict = False)
    
    path_truetime = './data/analysis/'
    truetime_up = pd.read_pickle(path_truetime + 'truetime_up_09' + date + '.pkl')
    truetime_up = truetime_up.loc[(
            ((truetime_up['station_status'] == '公交嘉定新城站') | (truetime_up['station_status'] == '菊园车站'))
            & ((truetime_up['max'] >= lolim) & (truetime_up['max'] <= hilim))
        )]
    truetime_up['intime'] = [False for _ in range(len(truetime_up))]
    truetime_up['lolim'] = [None for _ in range(len(truetime_up))]
    truetime_up['hilim'] = [None for _ in range(len(truetime_up))]
    
    truetime_down = pd.read_pickle(path_truetime + 'truetime_down_09' + date + '.pkl')
    truetime_down = truetime_down.loc[(
            ((truetime_down['station_status'] == '公交嘉定新城站') | (truetime_down['station_status'] == '菊园车站'))
            & ((truetime_down['max'] >= lolim) & (truetime_down['max'] <= hilim))
        )]
    truetime_down['intime'] = [False for _ in range(len(truetime_down))]
    truetime_down['lolim'] = [None for _ in range(len(truetime_down))]
    truetime_down['hilim'] = [None for _ in range(len(truetime_down))]
    
    truetime_up = truetime_up.apply(
        time_matching, 
        args=(timetable_up,),
        # result_type='broadcast',
        axis = 1,
        )
    truetime_down = truetime_down.apply(
        time_matching, 
        args=(timetable_down,), 
        # result_type='broadcast',
        axis = 1,
        )
    
    return truetime_up,truetime_down

In [ ]:
truetime_down = pd.read_pickle('./data/analysis/' + 'truetime_down_09' + '06' + '.pkl')
truetime_down.head(3)

In [ ]:
# timetable_up,timetable_down,truetime_up,truetime_down = generate_intime(date='06', time_range=[0, 0, 23, 59])
truetime_up,truetime_down = generate_intime(date='10', time_range=[0, 0, 23, 59])

In [ ]:
up_start_intime = truetime_up.loc[(truetime_up['station_status'] == '菊园车站') & (truetime_up['intime'])]
print(up_start_intime.intime.sum()/len(truetime_up.loc[truetime_up['station_status'] == '菊园车站']))

In [ ]:
down_start_intime = truetime_down.loc[(truetime_down['station_status'] == '公交嘉定新城站') & (truetime_down['intime'])]
print(down_start_intime.intime.sum()/len(truetime_down.loc[truetime_down['station_status'] == '公交嘉定新城站']))

In [ ]:
up_end_intime = truetime_up.loc[(truetime_up['station_status'] == '公交嘉定新城站') & (truetime_up['intime'])]
down_end_intime = truetime_up.loc[(truetime_down['station_status'] == '菊园车站') & (truetime_down['intime'])]

In [ ]:
from datetime import datetime as dt
first_start_timetable = pd.DataFrame()
last_start_timetable = pd.DataFrame()
first_start_direction_list = []
last_start_direction_list = []
first_start_date_list = []
last_start_date_list = []
first_start_time_list = []
last_start_time_list = []
for date in [6,
            7,
            8,
            9,
            10,
            ]:
    day = load_daydf(date)
    hilim = pd.Timestamp(dt(2021,9,date, hour=23,minute=59,second=59))
    lolim = pd.Timestamp(dt(2021,9,date, hour=4,minute=0,second=0))
    day = day.drop(day.loc[(day['time'] > hilim) | (day['time'] < lolim)].index,
                    axis = 0
                ).reset_index(drop=True)
    for direction in [0, 1]:
        if direction == 0:
            direc_str = 'up'
            start_timetable = day.loc[(day['direction']==direction) & (day['station_status']=='菊园车站')].groupby(['nidx','station_status'])['time'].agg(['max'])
        else:
            direc_str = 'down'
            start_timetable = day.loc[(day['direction']==direction) & (day['station_status']=='公交嘉定新城站')].groupby(['nidx','station_status'])['time'].agg(['max'])
        # if len(start_timetable)
        first_start_time_list.append(start_timetable.min())
        last_start_time_list.append(start_timetable.max())
        first_start_date_list.append(date)
        last_start_date_list.append(date)
        first_start_direction_list.append(direction)
        last_start_direction_list.append(direction)
        # print(str(date) + ':\n' + str(start_timetable.min()) + '\n' +  str(start_timetable.max()))
first_start_timetable = pd.DataFrame({'date':first_start_date_list,
                                        'direction':first_start_direction_list,
                                        'time':first_start_time_list})
last_start_timetable = pd.DataFrame({'date':last_start_date_list,
                                        'direction':last_start_direction_list,
                                        'time':last_start_time_list})


In [ ]:
real_up_timetable = pd.concat([first_start_timetable.loc[first_start_timetable['direction']==0], last_start_timetable.loc[last_start_timetable['direction']==0]])
real_down_timetable = pd.concat([first_start_timetable.loc[first_start_timetable['direction']==1], last_start_timetable.loc[last_start_timetable['direction']==1]])

In [ ]:
real_up_timetable

In [ ]:
real_down_timetable

In [ ]:
standard_first_start_timetable = pd.DataFrame()
standard_last_start_timetable = pd.DataFrame()
standard_first_start_direction_list = []
standard_last_start_direction_list = []
standard_first_start_date_list = []
standard_last_start_date_list = []
standard_first_start_time_list = []
standard_last_start_time_list = []
for date in ['06',
            '07',
            '08',
            '09',
            '10',
            ]:
    path = './data/timetable/'
    up = pd.read_csv(path + 'timetable_up_09'+date+'.csv')
    up = process_timetable(up)
    down = pd.read_csv(path + 'timetable_down_09'+date+'.csv')
    down = process_timetable(down)
    for i, real_up_row in real_up_timetable.iterrows():
        

In [ ]:
for i, real_time_row in first_start_timetable.iterrows():
    for j, standard_time_row in 

In [ ]:
up = process_timetable(up, int(date))
up

In [ ]:
up.start_time[0].split(':')

In [ ]:
real_timetable = day.loc[(day['direction']==0) & (day['station_status'] == '菊园车站')].groupby(['nidx','station_status'])['time'].agg(['min','max'])
real_timetable = real_timetable.sort_values('max',ascending = True)
real_timetable

In [ ]:
for i in range(len(day.nidx.unique())):
    marker = day.loc[day['nidx']==i]['deadheading'].reset_index(drop=True)
    if len(marker) > 1:
        if marker[0] == 1:
            print('\n dead heading')
        print(real_timetable.loc[i].sort_values('min'))